<a href="https://colab.research.google.com/github/sdossou/DSA_Agentic_RAG/blob/main/DSA_Agentic_RAG_LangChainv1_Duck.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DSA Agentic RAG Powered by LangChain

This notebook builds a DSA Agentic RAG Applications with LangChain.

The tools used are listed below:

1. LangChain - more specifically LCEL
2. LangGraph
3. LangSmith



## Dependencies

Installing relevant dependencies.

In [ ]:
!pip install -qU langchain langchain_openai langgraph arxiv duckduckgo-search

In [ ]:
!pip install -qU faiss-cpu pymupdf

## Environment Variables

Setting up API keys for openAI and LangSmith.

In [ ]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

OpenAI API Key:··········


In [ ]:
from uuid import uuid4

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"AIE1 - LangGraph - {uuid4().hex[0:8]}"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangSmith API Key: ")

LangSmith API Key: ··········


## Initialise a Simple Chain using LCEL


### Retrieval

Setting up a local retriever system that looks at Arxiv papers on the topic of the DSA.

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import ArxivLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

docs = ArxivLoader(query="Digital Services Act", load_max_docs=5).load()

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=350, chunk_overlap=50
)

chunked_documents = text_splitter.split_documents(docs)

faiss_vectorstore = FAISS.from_documents(
    documents=chunked_documents,
    embedding=OpenAIEmbeddings(model="text-embedding-3-small"),
)

retriever = faiss_vectorstore.as_retriever()

### Augmented

Creating the RAG prompt.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Use the following context to answer the user's query. If you cannot answer the question, please respond with 'I don't know'.

Question:
{question}

Context:
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

### Generation

Initialising the model using OpenAI's `gpt-3.5-turbo` model.

In [ ]:
from langchain_openai import ChatOpenAI

openai_chat_model = ChatOpenAI(model="gpt-3.5-turbo")

### LCEL RAG Chain

Building the RAG chain.

In [ ]:
from operator import itemgetter
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

retrieval_augmented_generation_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": rag_prompt | openai_chat_model, "context": itemgetter("context")}
)

Testing the Chain.

In [ ]:
await retrieval_augmented_generation_chain.ainvoke({"question" : "What is the Digital Services Act?"})

{'response': AIMessage(content='The Digital Services Act is a legal proposal launched by the European Commission as part of the platform governance regulation. It aims to regulate platform accountability and content moderation for companies above certain size thresholds.', response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 2526, 'total_tokens': 2563}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3bc1b5746c', 'finish_reason': 'stop', 'logprobs': None}),
 'context': [Document(page_content='applicable legal standards in the ﬁeld of labour and consumer law.\nCCS Concepts: • Social and professional topics →Governmental regulations.\nAdditional Key Words and Phrases: Legal Compliance API, Digital Services Act, social media platforms, monetization\nACM Reference Format:\nCatalina Goanta, Thales Bertaglia, and Adriana Iamnitchi. 2022. The Case for a Legal Compliance API for the Enforcement of the EU’s\nDigital Services Act on Social Media Platforms. In 202

## DSA RAG Application using LangGraph Tools
DSA RAG application allows a user to ask a question and receive an answer about the DSA .

LangGraph is a tool that leverages LangChain Expression Language to build coordinated multi-actor and stateful applications that includes cyclic behaviour.

This notebook uses the Arxiv and the Duck Duck Go Web Search tools.

##  Creating the Tool Belt

Installing the agent with a toolbelt to help answer questions and add external knowledge with Arxiv and the Duck Duck Go Web Search tools.

In [ ]:
from langchain_community.tools.ddg_search import DuckDuckGoSearchRun
from langchain_community.tools.arxiv.tool import ArxivQueryRun

tool_belt = [
    DuckDuckGoSearchRun(),
    ArxivQueryRun()
]

### Actioning with Tools

Setting up the ToolExecutor to run the process.

In [ ]:
from langgraph.prebuilt import ToolExecutor

tool_executor = ToolExecutor(tool_belt)

### Model

Setting up the OpenAI model.

In [ ]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(temperature=0)

Binding the LangChain formatted tools to the model in an OpenAI function calling format.

In [ ]:
from langchain_core.utils.function_calling import convert_to_openai_function

functions = [convert_to_openai_function(t) for t in tool_belt]
model = model.bind_functions(functions)

## Agent State

`coordinated multi-actor and stateful applications`

"stateful" means that we want to have some kind of object which we can pass around our application that holds information about what the current situation (state) is. Since our system will be constructed of many parts moving in a coordinated fashion, we want to be able to ensure that we have some commonly understood idea of that state.

LangGraph leverages a `StatefulGraph` which uses an `AgentState` object to pass information between the various nodes of the graph.

This `AgentState` object is stored in a `TypedDict` with the key `messages` and the value is a `Sequence` of `BaseMessages` that will be appended to whenever the state changes.

Let's think about a simple example to help understand exactly what this means:

1. We initialize our state object:
  - `{"messages" : []}`
2. Our user submits a query to our application.
  - New State: `HumanMessage(#1)`
  - `{"messages" : [HumanMessage(#1)}`
3. We pass our state object to an Agent node which is able to read the current state. It will use the last `HumanMessage` as input. It gets some kind of output which it will add to the state.
  - New State: `AgentMessage(#1, additional_kwargs {"function_call" : "WebSearchTool"})`
  - `{"messages" : [HumanMessage(#1), AgentMessage(#1, ...)]}`
4. We pass our state object to a "conditional node" which reads the last state to determine if we need to use a tool - which it can determine properly because of our provided object

In [ ]:
from typing import TypedDict, Annotated, Sequence
import operator
from langchain_core.messages import BaseMessage

class AgentState(TypedDict):
  messages: Annotated[Sequence[BaseMessage], operator.add]

## Graph Setup

Setting up the graph (nodes and edges) with the state, the tools and the LLM previously defined.

In [ ]:
from langgraph.prebuilt import ToolInvocation
import json
from langchain_core.messages import FunctionMessage

def call_model(state):
  messages = state["messages"]
  response = model.invoke(messages)
  return {"messages" : [response]}

def call_tool(state):
  last_message = state["messages"][-1]

  action = ToolInvocation(
      tool=last_message.additional_kwargs["function_call"]["name"],
      tool_input=json.loads(
          last_message.additional_kwargs["function_call"]["arguments"]
      )
  )

  response = tool_executor.invoke(action)

  function_message = FunctionMessage(content=str(response), name=action.tool)

  return {"messages" : [function_message]}

Defining 2 nodes:

- `call_model` is a node that calls the model
- `call_tool` is a node which calls a tool


In [ ]:
from langgraph.graph import StateGraph, END

workflow = StateGraph(AgentState)

workflow.add_node("agent", call_model)
workflow.add_node("action", call_tool)

Adding the entrypoint.

In [ ]:
workflow.set_entry_point("agent")

Setting up a "conditional edge" which will use the output state of a node to determine which path to follow.

Creating an edge where the origin node is the agent node and the destination node can be either the action node or the END (finish the graph).

The dictionary passed in as the third parameter (the mapping) should be created with the possible outputs of our conditional function in mind. In this case should_continue outputs either "end" or "continue" which are subsequently mapped to the action node or the END node.

In [ ]:
def should_continue(state):
  last_message = state["messages"][-1]

  if "function_call" not in last_message.additional_kwargs:
    return "end"

  return "continue"

workflow.add_conditional_edges(
    "agent",
    should_continue,
    {
        "continue" : "action",
        "end" : END
    }
)

Adding the edge which connects the action node to the agent node closing the loop or cycle.

In [ ]:
workflow.add_edge("action", "agent")

Compiling the workflow.

In [ ]:
app = workflow.compile()

## Using the Graph and Testing

Testing the graph with a question that will trigger the wikidata tool.

In [ ]:
from langchain_core.messages import HumanMessage

inputs = {"messages" : [HumanMessage(content="What companies do the Digital Services Act apply to? Provide their names. When does it apply to them?")]}

app.invoke(inputs)

{'messages': [HumanMessage(content='What companies do the Digital Services Act apply to? Provide their names. When does it apply to them?'),
  AIMessage(content='The Digital Services Act (DSA) applies to a wide range of companies that provide digital services in the European Union. Some of the key companies that the DSA applies to include:\n\n1. Social media platforms\n2. Online marketplaces\n3. Search engines\n4. Cloud services\n5. App stores\n\nThe DSA aims to regulate these companies and hold them accountable for the content and services they provide online. It sets out rules and obligations for these companies to ensure a safer online environment for users.\n\nThe DSA is expected to come into effect in late 2022 or early 2023. Once it is in effect, companies that fall under its scope will need to comply with the regulations outlined in the DSA.', response_metadata={'token_usage': {'completion_tokens': 148, 'prompt_tokens': 172, 'total_tokens': 320}, 'model_name': 'gpt-3.5-turbo', '

Let's look at what happened:

1. Our state object was populated with our request
2. The state object was passed into our entry point (agent node) and the agent node added an `AIMessage` to the state object and passed it along the conditional edge
3. The conditional edge received the state object, found the "function_call" `additional_kwarg`, and sent the state object to the action node
4. The action node added the response from the OpenAI function calling endpoint to the state object and passed it along the edge to the agent node
5. The agent node added a response to the state object and passed it along the conditional edge
6. The conditional edge received the state object, could not find the "function_call" `additional_kwarg` and passed the state object to END where we see it output in the cell above!


## Agentic RAG with LangGraph and LCEL

Creating our final agent.

Adding the RAG chain as a node.

Adding some pre/post processing steps.

In [ ]:
def convert_state_to_query(state_object):
  return {"question" : state_object["messages"][-1].content}

def convert_response_to_state(response):
  return {"messages" : [response["response"]]}

langgraph_node_rag_chain = convert_state_to_query | retrieval_augmented_generation_chain | convert_response_to_state

Testing the new chain.

In [ ]:
await langgraph_node_rag_chain.ainvoke(inputs)

{'messages': [AIMessage(content='The Digital Services Act applies to companies that meet certain size thresholds and provide core platform services, such as operating systems, web browsers, and social networking services. It imposes specific obligations on these companies related to platform accountability and content moderation. The Act aims to establish a new legal regime for platform liability and transparency with respect to illegal content. It focuses on compliance with Union law or the law of a Member State, irrespective of the specific subject matter or nature of that law.', response_metadata={'token_usage': {'completion_tokens': 93, 'prompt_tokens': 2551, 'total_tokens': 2644}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3bc1b5746c', 'finish_reason': 'stop', 'logprobs': None})]}

Adding the nodes including the newly built LCEL component as a node called `first_action`.

Using the private RAG set-up: if  deemed sufficient, it will return that response to the user; and if not it will augment the response with the other tools.

In [ ]:
rag_agent = StateGraph(AgentState)

rag_agent.add_node("agent", call_model)
rag_agent.add_node("action", call_tool)
rag_agent.add_node("first_action", langgraph_node_rag_chain)

Setting up the new entry point to be the RAG pipeline.

In [ ]:
rag_agent.set_entry_point("first_action")

Adding a conditional node ("is the question fully answered by the RAG pipeline?" - Yes or No).


It asks the following question: "Is this question fully answered by the response?"

It answers the following "Yes, it is fully answered", or "No, it is not fully answered".

This is implemented in the function below using Pydantic and GPT-4.



In [ ]:
from langchain.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.output_parsers.openai_tools import PydanticToolsParser
from langchain_core.utils.function_calling import convert_to_openai_tool

def is_fully_answered(state):

  ### Extract the question and response from the RAG pipeline
  question = state["messages"][0].content
  answer = state["messages"][-1].content

  ### Create a Pydantic model to capture the LLMs response
  class answered(BaseModel):
    binary_score: str = Field(description="Fully answered: 'yes' or 'no'")

  ### The reasoning model will ensure it can answer our question properly
  model = ChatOpenAI(model="gpt-4-turbo-preview", temperature=0)

  ### Create and bind the tool to the model
  answered_tool = convert_to_openai_tool(answered)

  model = model.bind(
      tools=[answered_tool],
      tool_choice={"type" : "function", "function" : {"name" : "answered"}}
  )

  ### Parse the output into a usable format
  parser_tool = PydanticToolsParser(tools=[answered])

  prompt = PromptTemplate(
      template="""You will determine if the question is fully answered by the response.\n
      Question:
      {question}

      Response:
      {answer}

      You will respond with either 'yes' or 'no'.""",
      input_variables=["question", "answer"])

  ### LCEL chain
  fully_answered_chain = prompt | model | parser_tool

  response = fully_answered_chain.invoke({"question" : question, "answer" : answer})

  if response[0].binary_score == "no":
    return "continue"

  return "end"

Adding the conditional edge.

In [ ]:
rag_agent.add_conditional_edges(
    "first_action",
    is_fully_answered,
    {
        "continue" : "agent",
        "end" : END
    }
)

Using the original prompt to determine if more tools are needed.

In [ ]:
def should_continue(state):
  last_message = state["messages"][-1]

  if "function_call" not in last_message.additional_kwargs:
    return "end"

  return "continue"

rag_agent.add_conditional_edges(
    "agent",
    should_continue,
    {
        "continue" : "action",
        "end" : END
    }
)

Defining the final edge.

In [ ]:
rag_agent.add_edge("action", "agent")

 Compiling the rag agent.

In [ ]:
rag_agent_app = rag_agent.compile()

Testing the RAG agent app.

In [ ]:
question = "What is the EU Digital Services Act?"

inputs = {"messages" : [HumanMessage(content=question)]}

rag_agent_app.invoke(inputs)

{'messages': [HumanMessage(content='What is the EU Digital Services Act?'),
  AIMessage(content='The EU Digital Services Act (DSA) is a regulatory proposal launched by the European Commission to address harms in the digital economy. It aims to facilitate digital enforcement and compliance with legal obligations on social media platforms.', response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 2600, 'total_tokens': 2641}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3bc1b5746c', 'finish_reason': 'stop', 'logprobs': None})]}

Testing with another example.

In [ ]:
question = "How does the Digital Services Act regulate misinformation?"

inputs = {"messages" : [HumanMessage(content=question)]}

rag_agent_app.invoke(inputs)

{'messages': [HumanMessage(content='How does the Digital Services Act regulate misinformation?'),
  AIMessage(content='The Digital Services Act (DSA) aims to establish a new legal regime for platform liability and transparency with respect to illegal content. It defines illegal content as any information that is not in compliance with Union law or the law of a Member State, irrespective of the precise subject matter or nature of that law. The DSA focuses on transparency in compliance and digital monitoring with new procedural obligations to regulate content posted online, including misinformation.', response_metadata={'token_usage': {'completion_tokens': 84, 'prompt_tokens': 2502, 'total_tokens': 2586}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3bc1b5746c', 'finish_reason': 'stop', 'logprobs': None})]}